Warp Radolan data to GTIFF 

In [6]:
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#Working dir
wdir = "/home/elisabeth/Dokumente/master/radolan_data/raw/"
file = "raa01-sf_10000-1701012350-dwd---bin"
#Read the original data: data = 2d Array(row,col) and metadata (productType, datetime,nodataflag)
#Origin in lower left corner
data_raw, meta  = wrl.io.read_radolan_composite(wdir + file)

#Round data to 2 decimal places to save memory
data_raw_s = np.around(data_raw, decimals = 2, out = None)

# Create Spatail Reference (Radolan projection)
proj_osr = wrl.georef.create_osr("dwd-radolan")

#Load EPSG 3857 
epsg_3857 = wrl.georef.epsg_to_osr(3857)

# Get projected RADOLAN coordinates for corner definition
#origin in lower left corner
xy_raw = wrl.georef.get_radolan_grid(900, 900)

#flip the data and the coords up-down (from lower left to upper left corner)
data, xy = wrl.georef.set_raster_origin(data_raw, xy_raw, 'upper')

#Create raster dataset: Geotiff with radolan projection
ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_osr)

#Reproject dataset to EPSG 3857
ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=epsg_3857, spacing=1000)
#Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
#This line is needed until fixed
ds2.SetProjection(epsg_3857.ExportToWkt())

#write raster dataset
filename = file[15:25] + ".tif"
wrl.io.write_raster_dataset(wdir + filename, ds2, "GTiff")
#

dest_src available
